In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score 
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier


In [2]:
train =  pd.read_csv("train.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [3]:
train = train.drop(["Loan_ID","LoanAmount","Loan_Amount_Term"],axis=1)
train["Dependents"] = train["Dependents"].replace("3+","3")
train.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [4]:
train["Gender"].fillna(train["Gender"].mode()[0],inplace=True)
train["Married"].fillna(train["Married"].mode()[0],inplace=True)
train["Self_Employed"].fillna(train["Self_Employed"].mode()[0],inplace=True)
train["Credit_History"].fillna(train["Credit_History"].mode()[0],inplace=True)
train["Dependents"].fillna(train["Dependents"].mode()[0],inplace=True)
train.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [5]:
train["Total_Income"] = train["ApplicantIncome"] + train["CoapplicantIncome"]
train.drop(["ApplicantIncome","CoapplicantIncome"],axis=1,inplace=True)
train.head()

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Loan_Status,Total_Income
0,Male,No,0,Graduate,No,1.0,Urban,Y,5849.0
1,Male,Yes,1,Graduate,No,1.0,Rural,N,6091.0
2,Male,Yes,0,Graduate,Yes,1.0,Urban,Y,3000.0
3,Male,Yes,0,Not Graduate,No,1.0,Urban,Y,4941.0
4,Male,No,0,Graduate,No,1.0,Urban,Y,6000.0


In [6]:
train["Credit_History"] = train["Credit_History"].astype(int)
train["Total_Income"] = train["Total_Income"].astype(int)
train.head()

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Loan_Status,Total_Income
0,Male,No,0,Graduate,No,1,Urban,Y,5849
1,Male,Yes,1,Graduate,No,1,Rural,N,6091
2,Male,Yes,0,Graduate,Yes,1,Urban,Y,3000
3,Male,Yes,0,Not Graduate,No,1,Urban,Y,4941
4,Male,No,0,Graduate,No,1,Urban,Y,6000


In [7]:
lc = LabelEncoder()
train["Gender"] = lc.fit_transform(train["Gender"])
train["Married"] = lc.fit_transform(train["Married"])
train["Self_Employed"] = lc.fit_transform(train["Self_Employed"])
train["Education"] = lc.fit_transform(train["Education"])
train["Property_Area"] = lc.fit_transform(train["Property_Area"])
train["Loan_Status"] = lc.fit_transform(train["Loan_Status"])
train["Dependents"] = train["Dependents"].astype('int64')
train["Credit_History"] = train["Credit_History"].astype('int64')
train["Total_Income"] = train["Total_Income"].astype('int64')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Gender          614 non-null    int32
 1   Married         614 non-null    int32
 2   Dependents      614 non-null    int64
 3   Education       614 non-null    int32
 4   Self_Employed   614 non-null    int32
 5   Credit_History  614 non-null    int64
 6   Property_Area   614 non-null    int32
 7   Loan_Status     614 non-null    int32
 8   Total_Income    614 non-null    int64
dtypes: int32(6), int64(3)
memory usage: 28.9 KB


In [8]:
train.head()


,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Loan_Status,Total_Income
0,1,0,0,0,0,1,2,1,5849
1,1,1,1,0,0,1,0,0,6091
2,1,1,0,0,1,1,2,1,3000
3,1,1,0,1,0,1,2,1,4941
4,1,0,0,0,0,1,2,1,6000


In [9]:
X = train.drop(['Loan_Status'], axis=1)
y = train['Loan_Status']

In [10]:
sc = MinMaxScaler(feature_range=(0, 1))
X_scaled = sc.fit_transform(X)

In [11]:
nbc = GaussianNB()
nbc.fit(X_scaled, y)

GaussianNB()

In [12]:
pickle.dump(nbc, open("model.sav", "wb"))
pickle.dump(sc, open("scaler.sav", "wb"))

In [13]:
xgb=XGBClassifier(random_state=42,learning_rate= 0.1)
xgb=xgb.fit(X_scaled,y)
pickle.dump(xgb, open("model1.sav", "wb"))

C:\Users\KARTO07\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[10:08:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [17]:
xgb_mod=GradientBoostingClassifier(random_state=42,learning_rate= 0.1)
xgb_mod=xgb_mod.fit(X_scaled,y) 
pickle.dump(xgb_mod, open("model2.sav", "wb"))

In [28]:
x_train,x_val,y_train,y_val = train_test_split(X,y,test_size=0.3)

In [29]:
nbc.fit(x_train,y_train)

GaussianNB()

In [30]:
pred = nbc.predict(x_val)
score = accuracy_score(y_val,pred)*100
score

82.70270270270271